In [1]:
import rtree
import fiona.crs
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [2]:
index = rtree.Rtree()
zones = gpd.read_file('NYCHA/NYCHA_developments_July2011.shp').to_crs(fiona.crs.from_epsg(2263))

In [38]:
f = pd.read_csv('cvh_people.csv')
df = pd.DataFrame(f)

,Internal Contact ID,Latitude,Longitude
0,5,40.687482,-73.963384
1,226,40.769909,-73.992111
2,228,40.746794,-73.982509
3,230,40.823687,-73.868344
4,232,40.760444,-73.973270


In [43]:
df['geometry'] = df.apply(lambda x: Point((float(x.Longitude), float(x.Latitude))), axis=1)

In [5]:
df = gpd.GeoDataFrame(df, geometry='geometry')

In [45]:
df.head()

,Internal Contact ID,Latitude,Longitude,geometry
0,5,40.687482,-73.963384,POINT (-73.96338399999999 40.687482)
1,226,40.769909,-73.992111,POINT (-73.99211099999999 40.76990900000001)
2,228,40.746794,-73.982509,POINT (-73.9825089 40.7467938)
3,230,40.823687,-73.868344,POINT (-73.86834350000001 40.8236871)
4,232,40.760444,-73.973270,POINT (-73.97327 40.760444)


In [47]:
points = df
points.crs = {'proj' : 'latlong', 'init': 'epsg:2263'}
zones = zones.to_crs({'proj' : 'latlong', 'init': 'epsg:2263'})

In [48]:
merged = gpd.sjoin(points, zones, how = 'inner', op = 'intersects')

In [49]:
merged.groupby("BOROUGH")["Internal Contact ID"].count()

BOROUGH
BRONX            1494
BROOKLYN          837
MANHATTAN        1493
QUEENS            224
STATEN ISLAND       9
Name: Internal Contact ID, dtype: int64

In [50]:
p = merged.geometry.iloc[89]


In [51]:
merged.head()

,Internal Contact ID,Latitude,Longitude,geometry,index_right,BOROUGH,CUR_UNIT11,DEVELOPMEN,NONRES_BLD,RES_BLDG,TDS_NUM,TOT_POP11
89,912,40.717385,-73.978243,POINT (-73.9782433 40.7173846),268,MANHATTAN,2194,BARUCH,1,17,060,5274
2453,4409,40.716899,-73.978211,POINT (-73.978211 40.716899),268,MANHATTAN,2194,BARUCH,1,17,060,5274
2538,4498,40.717448,-73.978255,POINT (-73.978255 40.717448),268,MANHATTAN,2194,BARUCH,1,17,060,5274
3455,5438,40.717093,-73.978296,POINT (-73.978296 40.717093),268,MANHATTAN,2194,BARUCH,1,17,060,5274
3590,5574,40.716322,-73.979423,POINT (-73.979423 40.7163223),268,MANHATTAN,2194,BARUCH,1,17,060,5274


In [8]:
# merged.groupby('DEVELOPMEN').count().sort_values('Internal C', ascending = False)

KeyError: 'Internal C'

In [164]:
merged.head()

,Internal C,Latitude,Longitude,geometry,index_right,BOROUGH,CUR_UNIT11,DEVELOPMEN,NONRES_BLD,RES_BLDG,TDS_NUM,TOT_POP11
89,912,40.717385,-73.978243,POINT (-73.9782433 40.7173846),268,MANHATTAN,2194,BARUCH,1,17,060,5274
2453,4409,40.716899,-73.978211,POINT (-73.978211 40.716899),268,MANHATTAN,2194,BARUCH,1,17,060,5274
2538,4498,40.717448,-73.978255,POINT (-73.978255 40.717448),268,MANHATTAN,2194,BARUCH,1,17,060,5274
3455,5438,40.717093,-73.978296,POINT (-73.978296 40.717093),268,MANHATTAN,2194,BARUCH,1,17,060,5274
3590,5574,40.716322,-73.979423,POINT (-73.979423 40.7163223),268,MANHATTAN,2194,BARUCH,1,17,060,5274


In [165]:
zones.head()

,BOROUGH,CUR_UNIT11,DEVELOPMEN,NONRES_BLD,RES_BLDG,TDS_NUM,TOT_POP11,geometry
0,VARIOUS,16,FHA REPOSSESSED HOUSES (GROUP III),0,18,213,40,(POLYGON ((-73.93559146071935 40.6594891659584...
1,VARIOUS,19,FHA REPOSSESSED HOUSES (GROUP IV),0,22,226,60,(POLYGON ((-73.92913654009608 40.6620874071540...
2,VARIOUS,40,FHA REPOSSESSED HOUSES (GROUP V),0,50,260,103,(POLYGON ((-73.92285468610149 40.6560250264297...
3,QUEENS,9,FHA REPOSSESSED HOUSES (GROUP VI),0,14,273,22,(POLYGON ((-73.77437108469485 40.6637481197118...
4,QUEENS,11,FHA REPOSSESSED HOUSES (GROUP VII),0,17,274,22,(POLYGON ((-73.76955112561127 40.6615154005149...
